In [6]:
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt

import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1

import numpy as np

from matplotlib import pyplot as plt

# Acquire the Quantum Rings Provider
qr_provider = QuantumRingsProvider()

In [3]:
backend = qr_provider.get_backend("scarlet_quantum_rings")
shots = 1024

In [117]:
def qft_cct(qc, q):
    """
    Forward QFT circuit 
    Args:
        qc (QuantumCircuit): The quantum circuit
        q (QuantumRegister): Target quantum registers
    Returns:
        None
    """
    n = len(q)
    for i in reversed(range(n)):
        qc.h(q[i])
        for j in range(i):
            angle = np.pi / 2**(i - j)
            qc.cu1(angle, q[j], q[i])
    qc.barrier()

In [56]:
def iqft_cct(qc, q):
    """
    The inverse QFT circuit
    Args:
        qc (QuantumCircuit): The quantum circuit
        q (QuantumRegister): Target quantum registers
    Returns:
        None
    """
    n = len(q)
    for i in range(n):
        for j in range (1, i+1):
            angle = np.pi / 2**(i-j+1)
            qc.cu1(-angle, q[j-1], q[i])
        qc.h(q[i])
    qc.barrier()

In [115]:
def adder_reg(qc, q, a):
    """
    Addition of q+a stored on registers 'q' and 'a' using a QFT-based adder.
    Assumes the QFT basis.
    Args:
        qc (QuantumCircuit): The circuit
        q (QuantumRegister): Register to apply the adder
        a (QuantumRegister): Register to add by
    """
    n = len(q)
    for i in range(len(a)):
        for j in range(n-i):
            angle = np.pi / 2**(n - i - j - 1)
            qc.cu1(angle, a[i], q[j])
    qc.barrier()

In [116]:
def adder(qc, q, N, subtract=False):
    """
    Subtracts N from q stored on registers 'q'.
    Assumes QFT basis.
    Args:
        qc (QuantumCircuit): The circuit
        q (QuantumRegister): Register to apply the adder
        N (Integer): Integer to subtract by
        subtract (Boolean): Set to subtract rather than add
    """
    rotate_dir = 1
    if subtract: rotate_dir = -1
    n = len(q)
    N_bin = format(N, f'0{n}b')[::-1]
    for i in range(n):
        if N_bin[i] == '1':
            for j in range(i + 1):
                angle = np.pi / 2**(i - j)
                qc.p(rotate_dir * angle, q[j])
    qc.barrier()

In [118]:
def mod_adder(qc, q, a, N, ancilla):
    """
    Addition of q+a stored on registers 'q' and 'a' using a QFT-based adder.
    Args:
        qc (QuantumCircuit): The circuit
        q (QuantumRegister): Register to apply the adder
        a (QuantumRegister): Register to add by
        N: Modulus
        ancilla (QuantumRegister): Overflow control
    """
    qft_cct(qc, q)
    adder(qc, q, a)
    subtracter(qc, q, N)
    iqft_cct(qc, q)
    qc.cx(q[0], ancilla)

    # TODO: UNDERFLOW PROTECTION
    qc1 = QuantumCircuit(q, a, ancilla)
    adder(qc1, q, a)
    cadder = qc1.to_gate().control(1)
        

In [119]:
def mod_mul(qc, q, a):
    """
    Modular multiply by 'a': computes (a * x mod N)
    """
    # TODO
       

In [74]:
# Registers to store binary rep of a in a^x. For example, 7^x.
# Use as many as necessary to store your trial value of a.
adders = QuantumRegister(4, 'a')

# Registers to store the results of controlled exponentiation.
# Use as many as necessary to store N.
num_workers = int(np.ceil(np.log2(N)))
workers = QuantumRegister(num_workers, 'x')

# Registers to store binary reps of x in a^x. 
# Use as many as necessary to store the expected value of r, usually N.
# Can use fewer with some loss of precision.

#err = 0.25
#num_controls = 2*num_workers + int(np.ceil(np.log2(2 + 1/(2*err))))
num_controls = 3 # We know that 4 is the largest period of 7 in 15.
controllers = QuantumRegister(num_controls, 'c')
readout = ClassicalRegister(num_controls, 'r')

In [80]:
qc = QuantumCircuit(adders, workers, controllers, readout)

In [81]:
# Initialize source and target registers
qc.h(controllers)
qc.x(workers[-1])
qc.barrier()
qc.draw()

░ 
a_0: ──────░─
           ░ 
a_1: ──────░─
           ░ 
a_2: ──────░─
           ░ 
a_3: ──────░─
           ░ 
x_0: ──────░─
           ░ 
x_1: ──────░─
           ░ 
x_2: ──────░─
     ┌───┐ ░ 
x_3: ┤ X ├─░─
     ├───┤ ░ 
c_0: ┤ H ├─░─
     ├───┤ ░ 
c_1: ┤ H ├─░─
     ├───┤ ░ 
c_2: ┤ H ├─░─
     └───┘ ░ 
r: 3/════════

In [120]:
# Modular exponentiation via addition

In [104]:
# Calculate GCD w/ Euclid's Algorithm
def postprocess(a, r, N):
    l = a**r - 1
    gcd = euclids_gcd(l, N)
    if (gcd > 1) and (gcd < N):
        return gcd
    u = a**r + 1
    gcd = euclids_gcd(u, N)
    if (gcd > 1) and (gcd < N):
        return gcd
    return -1

def euclids_gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

a = 5
b = 15
print(f"The GCD of {a} and {b} is {euclids_gcd(a, b)}")

The GCD of 5 and 15 is 5
